## Extracción

In [27]:
# 🔧 Importar librerías necesarias
import pandas as pd
import requests
import numpy as np


# 🌐 URL de la API (archivo JSON alojado en GitHub)
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# 📥 Obtener los datos de la API
response = requests.get(url)
data = response.json()

# 🧾 Convertir a DataFrame
df = pd.DataFrame(data)

# 👀 Vista previa de los datos
df.head()


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


## Transformación

In [28]:
# Explorar las columnas del dataset y verificar sus tipos de datos.
# Ver columnas disponibles
print("Columnas del dataset:")
print(df.columns.tolist())

#Consultar el diccionario para comprender mejor el significado de las variables.
# Tipos de datos por columna
print("\nTipos de datos:")
print(df.dtypes)

# Ver el diccionario dentro del notebook (opcional)
with open("TelecomX_diccionario.md", "r", encoding="utf-8") as file:
    print(file.read())

#Identificar las columnas más relevantes para el análisis de evasión.
# Revisión de un ejemplo completo para entender la estructura de columnas anidadas
df.iloc[0]



Columnas del dataset:
['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']

Tipos de datos:
customerID    object
Churn         object
customer      object
phone         object
internet      object
account       object
dtype: object
#### Diccionario de datos

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción ad

customerID                                           0002-ORFBO
Churn                                                        No
customer      {'gender': 'Female', 'SeniorCitizen': 0, 'Part...
phone            {'PhoneService': 'Yes', 'MultipleLines': 'No'}
internet      {'InternetService': 'DSL', 'OnlineSecurity': '...
account       {'Contract': 'One year', 'PaperlessBilling': '...
Name: 0, dtype: object

## Comprobación de incoherencias

In [29]:
# 📌 Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())
print("\n")

# 📌 Verificar filas duplicadas
# Convertimos columnas con dicts a string
df_temp = df.copy()
for col in df_temp.columns:
    if df_temp[col].apply(lambda x: isinstance(x, dict)).any():
        df_temp[col] = df_temp[col].astype(str)

print("Cantidad de filas duplicadas:", df_temp.duplicated().sum())
print("\n")

# 📌 Verificar tipos únicos en columnas categóricas (seguro con dicts)
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"Valores únicos en '{col}':")
        
        # Evitar error si hay diccionarios en la columna
        if df[col].apply(lambda x: isinstance(x, dict)).any():
            print("⚠️ Contiene diccionarios, no se puede mostrar valores únicos directamente.")
        else:
            print(df[col].unique())
        
        print("—" * 50)


Valores nulos por columna:
customerID    0
Churn         0
customer      0
phone         0
internet      0
account       0
dtype: int64


Cantidad de filas duplicadas: 0


Valores únicos en 'customerID':
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
——————————————————————————————————————————————————
Valores únicos en 'Churn':
['No' 'Yes' '']
——————————————————————————————————————————————————
Valores únicos en 'customer':
⚠️ Contiene diccionarios, no se puede mostrar valores únicos directamente.
——————————————————————————————————————————————————
Valores únicos en 'phone':
⚠️ Contiene diccionarios, no se puede mostrar valores únicos directamente.
——————————————————————————————————————————————————
Valores únicos en 'internet':
⚠️ Contiene diccionarios, no se puede mostrar valores únicos directamente.
——————————————————————————————————————————————————
Valores únicos en 'account':
⚠️ Contiene diccionarios, no se puede mostrar valores únicos directament

## Manejo de inconsistencias

In [30]:
# Reemplazar valores vacíos en 'Churn' por NaN
df['Churn'] = df['Churn'].replace('', np.nan)

# Eliminar filas con 'Churn' faltante (opcional, si son pocas)
df = df.dropna(subset=['Churn'])

# Expandir columnas con diccionarios
customer_df = df['customer'].apply(pd.Series)
phone_df = df['phone'].apply(pd.Series)
internet_df = df['internet'].apply(pd.Series)
account_df = df['account'].apply(pd.Series)

# Combinar con el DataFrame original
df = pd.concat([df.drop(columns=['customer', 'phone', 'internet', 'account']),
                customer_df, phone_df, internet_df, account_df], axis=1)

# Comprobamos que ya no existan valores problemáticos
print(df.isnull().sum())
print(df.dtypes)


customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges             0
dtype: int64
customerID          object
Churn               object
gender              object
SeniorCitizen        int64
Partner             object
Dependents          object
tenure               int64
PhoneService        object
MultipleLines       object
InternetService     object
OnlineSecurity      object
OnlineBackup        object
DeviceProtection    object
TechSupport         object
StreamingTV         object
StreamingMovies     object
Contract            object
PaperlessBilling    object
PaymentMethod       object
Charges             object
dtype: 

In [32]:
# Eliminar caracteres como '$' y ',' antes de convertir a número
df['Charges'] = df['Charges'].astype(str).str.replace(r'[^0-9.]', '', regex=True)

# Convertir a numérico (NaN si no se puede convertir)
df['Charges'] = pd.to_numeric(df['Charges'], errors='coerce')

# Eliminar filas con valores faltantes en 'Charges'
df = df.dropna(subset=['Charges'])

# Crear la columna 'Cuentas_Diarias' dividiendo entre 30
df['Cuentas_Diarias'] = df['Charges'] / 30

# Verificar los primeros valores para ambas columnas
print(df[['Charges', 'Cuentas_Diarias']].head())

# Mostrar los primeros 10 valores de la columna Charges
print(df['Charges'].head(10))

# Mostrar todos los valores únicos para verificar la conversión
print(df['Charges'].unique())


       Charges  Cuentas_Diarias
40   75.751929         2.525064
44   61.616110         2.053870
48   99.253532         3.308451
80   45.811470         1.527049
104  19.955800         0.665193
40     75.751929
44     61.616110
48     99.253532
80     45.811470
104    19.955800
144    79.247650
151    92.656733
188    50.225540
248    39.553730
290    99.754036
Name: Charges, dtype: float64
[ 75.751929  61.61611   99.253532  45.81147   19.9558    79.24765
  92.656733  50.22554   39.55373   99.754036  20.051036  45.32145
  83.55435  100.56029  100.352799  42.12962   25.9135    97.955867
  19.85252   98.05713   95.095    100.4715    99.04135   18.55689
  85.72067  102.251359  75.25242   89.13342  109.65953   20.351267
  19.41083  105.82998   39.11309   73.652    100.554304  20.054
  21.1741   110.755832 103.46603   56.05      19.152     19.019
  80.08      51.052066  86.755186  45.8581    73.853371  81.45912
  25.15553   97.555598  85.085     45.2492    76.6893    69.01108
  79.52665   48.

In [33]:
# 1. Convertir valores textuales 'Sí' y 'No' en valores binarios (1 y 0)
cols_binarias = ['Churn', 'Partner', 'Dependents', 'PhoneService', 
                 'PaperlessBilling']

for col in cols_binarias:
    df[col] = df[col].map({'Sí': 1, 'No': 0})

# 2. Normalizar valores como 'Male' / 'Female' o traducir al español si es necesario
df['gender'] = df['gender'].map({'Male': 'Masculino', 'Female': 'Femenino'})

# 3. Renombrar columnas para mayor claridad (si se requiere traducir a español)
df = df.rename(columns={
    'gender': 'Genero',
    'SeniorCitizen': 'Mayor_65',
    'Partner': 'Tiene_Pareja',
    'Dependents': 'Tiene_Dependientes',
    'tenure': 'Meses_Contrato',
    'PhoneService': 'Servicio_Telefono',
    'MultipleLines': 'Lineas_Multiples',
    'InternetService': 'Servicio_Internet',
    'OnlineSecurity': 'Seguridad_En_Linea',
    'OnlineBackup': 'Respaldo_En_Linea',
    'DeviceProtection': 'Proteccion_Dispositivo',
    'TechSupport': 'Soporte_Tecnico',
    'StreamingTV': 'Television_Streaming',
    'StreamingMovies': 'Peliculas_Streaming',
    'Contract': 'Tipo_Contrato',
    'PaperlessBilling': 'Factura_Electronica',
    'PaymentMethod': 'Metodo_Pago',
    'Charges': 'Cargos_Mensuales',
    'Cuentas_Diarias': 'Cargos_Diarios'
})

# 4. Verificamos el resultado
print(df.head())


     customerID  Churn     Genero  Mayor_65  Tiene_Pareja  Tiene_Dependientes  \
40   0064-YIJGF    0.0  Masculino         0           NaN                 NaN   
44   0074-HDKDG    0.0  Masculino         0           NaN                 NaN   
48   0080-OROZO    0.0   Femenino         0           0.0                 0.0   
80   0128-MKWSG    0.0   Femenino         0           0.0                 NaN   
104  0178-CIIKR    0.0   Femenino         0           0.0                 0.0   

     Meses_Contrato  Servicio_Telefono  Lineas_Multiples Servicio_Internet  \
40               27                NaN               Yes       Fiber optic   
44               25                NaN                No               DSL   
48               35                NaN                No       Fiber optic   
80               26                0.0  No phone service               DSL   
104               3                NaN                No                No   

     ...    Respaldo_En_Linea Proteccion_Dis

#📊 Carga y análisis

#📄Informe final